## Libraries

In [1]:
import pandas as pd
from pandas import DataFrame 
import numpy as np

In [2]:
data_df = pd.read_csv(r'movies_metadata.csv')

#data_df = data_df.head(20000)
#display(data_df)


C:\Users\Augustin\AppData\Local\Temp\ipykernel_8744\3378795966.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(r'movies_metadata.csv')


## Prepocess the data 

In [3]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

data_df = data_df.fillna('')
data_df = data_df.drop(['adult', 'belongs_to_collection', 'budget', 'homepage', 'imdb_id', 'original_title', 'poster_path', 'production_companies', 'production_countries', 'spoken_languages', 'status', 'tagline', 'video', 'vote_count'], axis=1)
#data_df = data_df.dropna()
print(data_df.info())

stop_words = set(stopwords.words('english'))

def preprocess_title(title):
    title = title.translate(str.maketrans('', '', string.punctuation)).lower()
    words = word_tokenize(title)
    words = [word for word in words if not word in stop_words]
    return ' '.join(words)
data_df['preprocessed_title'] = data_df['title'].apply(preprocess_title)
data_df.to_excel('testxls.xls')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   genres             45466 non-null  object
 1   id                 45466 non-null  object
 2   original_language  45466 non-null  object
 3   overview           45466 non-null  object
 4   popularity         45466 non-null  object
 5   release_date       45466 non-null  object
 6   revenue            45466 non-null  object
 7   runtime            45466 non-null  object
 8   title              45466 non-null  object
 9   vote_average       45466 non-null  object
dtypes: object(10)
memory usage: 3.5+ MB
None


C:\Users\Augustin\AppData\Local\Temp\ipykernel_8744\266579804.py:18: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  data_df.to_excel('testxls.xls')


## Show Duplicate movies

In [4]:
# iterate through the movie titles and add release dates to duplicates
for index, row in data_df.iterrows():
    movie_title = row['title']
    release_date = row['release_date']
    duplicates = data_df[data_df['title'] == movie_title]
    
    if len(duplicates) > 1:
        for dup_index, dup_row in duplicates.iterrows():
            data_df.at[dup_index, 'Release Date Duplicate'] = release_date

# show the resulting dataframe
print(data_df)

                                                  genres      id  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...     862   
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...    8844   
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   15602   
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   31357   
4                         [{'id': 35, 'name': 'Comedy'}]   11862   
...                                                  ...     ...   
45461  [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...  439050   
45462                      [{'id': 18, 'name': 'Drama'}]  111109   
45463  [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...   67758   
45464                                                 []  227506   
45465                                                 []  461257   

      original_language                                           overview  \
0                    en  Led by Woody, Andy's toys live happily in his ...   
1                    en  Wh

## Compute the TF-IDF matrix

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_df['title'])

## Compute the cosine similarity matrix

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

## Define a function that returns the top 5 most similar movies

In [7]:
def get_top_similar_movies(movie_index, cosine_sim_matrix, df):
    sim_scores = list(enumerate(cosine_sim_matrix[movie_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return df.iloc[movie_indices]['title']

In [8]:
def get_top_5_similar_movies(row:pd.Series) -> pd.Series:
    """ 
    this function will return the 5 most similar movies than the one in te given row
    """
    # Get the similarity scores for the current movie
    similarity_scores = cosine_sim_matrix[row.name]

    # Get the indices of the top 5 similar movies -> (6 because the movie of the line row will be included) -> in return select only the 5 last ones
    top_5_indices = np.argpartition(-similarity_scores, range(6))[:6]
    #get the ids of the movies (ids to be sure it is unique) (duplicates in movie title)
    top_5_ids = [data_df.iloc[i]['title'] for i in top_5_indices]

    return pd.Series(top_5_ids[1:], index=['sim_movie_1', 'sim_movie_2', 'sim_movie_3', 'sim_movie_4', 'sim_movie_5'])


In [9]:
data_df[['sim_movie_1', 'sim_movie_2', 'sim_movie_3', 'sim_movie_4', 'sim_movie_5']] = data_df.apply(get_top_5_similar_movies, axis=1)

## Print the top 5 similar movies

In [ ]:
"""
for i, row in data_df.iterrows():
    print('Top 5 similar movies for', row['title'], ':')
    print(get_top_similar_movies(i, cosine_sim_matrix, data_df))
    print('\n')
"""

"\nfor i, row in data_df.iterrows():\n    print('Top 5 similar movies for', row['title'], ':')\n    print(get_top_similar_movies(i, cosine_sim_matrix, data_df))\n    print('\n')\n"

## Save Data in CSV file

In [10]:
data_df.to_csv('raw_data1.csv', index=False)
data_df.to_excel('raw_data1.xls', index=False)

C:\Users\Augustin\AppData\Local\Temp\ipykernel_8744\1357900905.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  data_df.to_excel('raw_data1.xls', index=False)


In [ ]:
df_test = pd.read_csv('raw_data.csv')
df_test.head()

C:\Users\Augustin\AppData\Local\Temp\ipykernel_17028\873730437.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('raw_data.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,toy story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,grumpier old men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,waiting exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,father bride part ii,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
